In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.manifold import TSNE

import plotly.express as px
import matplotlib.pyplot as plt

from var import DATA_OUT, MODEL_CATB, IMAGE_OUT, FORECAST_HOURS_IN_ADVANCE

In [ ]:
df = pd.read_pickle(Path(DATA_OUT, 'df_dataset.pickle'))

df[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'] = df[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].replace(
    {True: 1, False: 0}
)

In [ ]:
X = df[
    [
        'ie_fix',
        'iu_fix',
        'hf',
        'solar_zenith_angle',
        f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h',
    ]
].copy()

In [ ]:
X_ = X.sample(frac=0.25).copy()

In [ ]:
X_.loc[X['hf'].isna(),'hf'] = 0

In [ ]:
X_sc = MinMaxScaler().fit_transform(X_.values)

In [ ]:
n_comps = 2

tsne = TSNE(n_components=n_comps, perplexity=60, random_state=42, metric='euclidean')
projections = tsne.fit_transform(X_sc)

if n_comps == 3:
    fig = px.scatter_3d(
        x=projections[:,0],
        y=projections[:,1],
        z=projections[:,2],
        color=X_sc[:,-1],
        color_continuous_scale=px.colors.qualitative.Plotly
    )
    fig.show(autosize=False)
elif n_comps == 2:
    fig = px.scatter(
        x=projections[:,0],
        y=projections[:,1],
        color=X_sc[:,-1],
    )
    fig.show(autosize=False)

In [ ]:
# fig.write_html(
#     Path(IMAGE_OUT, 'tSNE_labels.html')
# )